<a href="https://colab.research.google.com/github/mel-zheng/text-generator/blob/main/test_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install -q torch

In [3]:
!pip install -q transformers

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 49.9 MB/s 
     |████████████████████████████████| 596 kB 62.8 MB/s 
     |████████████████████████████████| 6.6 MB 33.6 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 


In [4]:
!pip install -q panel

In [5]:
import numpy as np
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from random import choice

In [6]:
tok = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [7]:
def get_pred(text, model, tok, p=0.7):
    input_ids = torch.tensor(tok.encode(text)).unsqueeze(0)
    logits = model(input_ids)[0][:, -1]
    probs = F.softmax(logits, dim=-1).squeeze()
    idxs = torch.argsort(probs, descending=True)
    res, cumsum = [], 0.
    for idx in idxs:
        res.append(idx)
        cumsum += probs[idx]
        if cumsum > p:
            pred_idx = idxs.new_tensor([choice(res)])
            break
    pred = tok.convert_ids_to_tokens(int(pred_idx))
    return tok.convert_tokens_to_string(pred)

In [23]:
get_pred("It's an amazing", model, tok, p=0.7)

' thought'

In [25]:
import panel as pn
pn.extension() # loading panel's extension for jupyter compatibility 
text_input = pn.widgets.TextInput()
generated_text = pn.pane.Markdown(object=text_input.value)
text_input.link(generated_text, value='object')

Watcher(inst=TextInput(), cls=<class 'panel.widgets.input.TextInput'>, fn=<function Reactive.link.<locals>.link at 0x7f62e872acb0>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

In [26]:
def click_cb(event):
    pred = get_pred(generated_text.object, model, tok)
    generated_text.object += pred

In [27]:
button = pn.widgets.Button(name="Generate",button_type="primary")
button.on_click(click_cb)

In [28]:
app = pn.Column(text_input, button, generated_text); app


Column
    [0] TextInput()
    [1] Button(button_type='primary', name='Generate')
    [2] Markdown(str)